In [ ]:
# Load ML forecasting dataset
query = """
SELECT 
    cntr_code,
    year,
    total_energy_gwh,
    renewable_energy_gwh,
    fossil_energy_gwh,
    pct_renewable,
    pct_fossil,
    yoy_change_total_energy_pct,
    yoy_change_renewable_pct,
    energy_density_gwh_per_km2,
    area_km2
FROM ml_dataset_forecasting_v1
ORDER BY cntr_code, year
"""

df = pd.read_sql(query, engine)
print(f"✅ Loaded {len(df)} records")
print(f"Countries: {df['cntr_code'].nunique()}")
print(f"Years: {df['year'].min()} - {df['year'].max()}")
df.head()

In [ ]:
# Focus on 2016-2019 period
df_2016_2019 = df[df['year'].between(2016, 2019)].copy()

# Calculate YoY changes manually for verification
df_2016_2019 = df_2016_2019.sort_values(['cntr_code', 'year'])
df_2016_2019['prev_year_total'] = df_2016_2019.groupby('cntr_code')['total_energy_gwh'].shift(1)
df_2016_2019['manual_yoy_pct'] = ((df_2016_2019['total_energy_gwh'] - df_2016_2019['prev_year_total']) / df_2016_2019['prev_year_total'] * 100)

# Identify extreme YoY changes in 2018
extreme_yoy = df_2016_2019[
    (df_2016_2019['year'] == 2018) & 
    (df_2016_2019['yoy_change_total_energy_pct'].abs() > 100)
].sort_values('yoy_change_total_energy_pct', ascending=False)

print("🔍 Countries with extreme YoY changes in 2018 (>100%):")
print(extreme_yoy[['cntr_code', 'year', 'total_energy_gwh', 'prev_year_total', 
                   'yoy_change_total_energy_pct', 'pct_renewable']].to_string())

# Visualize
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Plot 1: Total energy over time for countries with extreme changes
ax1 = axes[0]
for country in extreme_yoy['cntr_code'].head(10):
    country_data = df_2016_2019[df_2016_2019['cntr_code'] == country]
    ax1.plot(country_data['year'], country_data['total_energy_gwh'], 
             marker='o', label=country, linewidth=2)
ax1.set_xlabel('Year')
ax1.set_ylabel('Total Energy (GWh)')
ax1.set_title('Total Energy Consumption - Countries with Extreme 2018 YoY Changes')
ax1.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
ax1.grid(True, alpha=0.3)

# Plot 2: YoY change percentage
ax2 = axes[1]
yoy_2018 = df_2016_2019[df_2016_2019['year'] == 2018].sort_values('yoy_change_total_energy_pct', ascending=False)
ax2.barh(yoy_2018['cntr_code'].head(15), yoy_2018['yoy_change_total_energy_pct'].head(15))
ax2.set_xlabel('YoY Change (%)')
ax2.set_title('Year-over-Year Change in 2018')
ax2.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()